# Ensemble - WBF
utils 폴더안에 해당 주피터파일 위치해서 실행하기

In [1]:
import pandas as pd
import numpy as np
from pycocotools.coco import COCO
import os
import cv2
import json
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from weighted_boxes_fusion import *

/tmp/ipykernel_2202403/1171383838.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
test_folder = "../../data/medical/img/test/"
get_img_files = os.listdir(test_folder)
# if get_img_files[-1].startswith('.'):  # '.'으로 시작하는 파일 제거용
#     get_img_files.pop(-1)

In [3]:
len(get_img_files) # 100개 뜨면 정상

100

## COCO format test.json 만들기

In [7]:
# coco_data = {
#         "images": [],
#         "annotations": [],
#         "categories": [{"id": 1, "name": "text"}],    # 이부분은 비워놔도 됨(only text)
# }

# for i, test_file in enumerate(get_img_files):
#     img = cv2.imread(test_folder+test_file)
#     h, w, c = img.shape
#     coco_image = {
#             "id": i,
#             "width": w,
#             "height": h,
#             "file_name": test_file,
#             "license": 0, 
#             "flickr_url": None, 
#             "coco_url": None, 
#             "date_captured": None
#     }
#     coco_data["images"].append(coco_image)

# with open('../../data/medical/ufo/test.json', 'w') as f:
#     json.dump(coco_data, f)

## 앙상블 시작
code/utils/ensemble/~.json 파일들 두기

#### recipe 1
- recipe1 : 노이즈에 취약, 후라이 까는거 보정

In [6]:
submission_files = ["output200_kim.json", "output250_kim.json", "output300_kang.json"]

#### recipe 2
- recipe2 : 노이즈 개선(FP 감소), FN 증가(+세로박스) -> FN 증가 이유는 합친 모델이 2개라서 둘 중에 하나라도 안되어 있으면 없앰.

In [11]:
submission_files = ["output200_kim.json", "output300_kang.json"]

#### recipe 3
- recipe3 : recipe2에서 FN 개선(세로 박스도 무난히 잡음) -> recipe1에서 간간히 큐알 코드를 검출하던 문제가 해결됨.

In [6]:
submission_files = ["output200_kim.json", "output300_kang.json", "output165_lee.json"]

#### recipe 4
- recipe3 대비 노이즈에 조금 더 취약해짐, FN 또한 살짝 증가된 것으로 보임

In [10]:
submission_files = ["output240_messi.json", "output300_kang.json", "output165_lee.json"]

#### recipe 5
- recipe4에서 증가되었던 FN이 개선됨. 하지만 노이즈는 개선이 되지 않아서 노이즈를 개선 시켜줄 모델이 하나 더 필요함.

In [5]:
submission_files = ["output200_kim.json", "output240_messi.json", "output300_kang.json", "output165_lee.json"]

#### recipe 6
- recipe5의 문제였던 노이즈 개선됨. 하지만 개선된 FN도 아주 살짝 증가함.

In [10]:
submission_files = ["output200_kim.json", "output240_messi.json", "output165_lee.json"]

#### recipe 7
- recipe5에서 output160_lee.json -> output180_lee.json으로 변경

In [5]:
submission_files = ["output200_kim.json", "output240_messi.json", "output300_kang.json", "output180_lee.json"]

#### recipe 8
- recipe6에서 output160_lee.json -> output180_lee.json으로 변경

In [10]:
submission_files = ["output200_kim.json", "output240_messi.json", "output180_lee.json"]

In [5]:
annotation = '../../data/medical/ufo/test.json'
coco = COCO(annotation)

iou_thr = 0.5            # TODO : iou threshold 설정
skip_box_thr = 0.0001    # TODO : skip iou threshold 설정 0.0001

images_anno = {}
for i, test_file in enumerate(get_img_files):
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for submission in submission_files:
        with open('./ensemble/'+submission, 'r') as f:
            data = json.load(f)
            images = data['images']
        words = images[image_info['file_name']]
        box_list = []
        for word_id, word_data in words["words"].items():
            # UFO to PascalVOC format
            [tl, tr, br, bl] = word_data["points"]
            xmin = min(tl[0], tr[0], br[0], bl[0])
            ymin = min(tl[1], tr[1], br[1], bl[1])
            xmax = max(tl[0], tr[0], br[0], bl[0])
            ymax = max(tl[1], tr[1], br[1], bl[1])
            box = [xmin/image_info['width'], ymin/image_info['height'], xmax/image_info['width'], ymax/image_info['height']]
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append([1.0]*len(words["words"].items()))
        labels_list.append([1]*len(words["words"].items()))

    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        prediction_words = []
        points_list = []
        for idx, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            # PascalVOC to UFO format
            p_xmin = box[0]*image_info['width']
            p_ymin = box[1]*image_info['height']
            p_xmax = box[2]*image_info['width']
            p_ymax = box[3]*image_info['height']
            p_width = p_xmax - p_xmin
            p_height = p_ymax - p_ymin
            p_tl = [p_xmin, p_ymin]
            p_tr = [p_xmin + p_width, p_ymin]
            p_br = [p_xmin + p_width, p_ymin + p_height]
            p_bl = [p_xmin, p_ymin + p_height]
            points = [p_tl, p_tr, p_br, p_bl]
            points_list.append(points)
        prediction_words = {idx: dict(points=p) for idx, p in enumerate(points_list)}
    images_anno[image_info['file_name']] = dict(words=prediction_words)

images_anno = {'images':images_anno}
# 앙상블 결과 출력용
with open('ensemble/result.json', 'w') as f:
    json.dump(images_anno, f)

# 제출용 : csv 파일로 바로 변경
with open('ensemble/result.csv', 'w') as f:
    json.dump(images_anno, f, indent=4)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/data/ephemeral/home/level2-cv-datacentric-cv-04/CSH/utils/wbf_ensemble.py:45: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn("X1 < 0 in box. Set it to 0.")
/data/ephemeral/home/level2-cv-datacentric-cv-04/CSH/utils/wbf_ensemble.py:61: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn("Y1 < 0 in box. Set it to 0.")
/data/ephemeral/home/level2-cv-datacentric-cv-04/CSH/utils/wbf_ensemble.py:56: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn(
/data/ephemeral/home/level2-cv-datacentric-cv-04/CSH/utils/wbf_ensemble.py:72: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn(


## 앙상블 결과 출력해보기

In [7]:
root_path = '../../data/medical/img/test/'
anno_root = './ensemble/result.json'

bboxes = []

with open(anno_root, 'r') as f:
    train_json = json.load(f)
    images = train_json['images']
    images_df = pd.DataFrame.from_dict(images)

image_id = list(images_df)
fnames = [os.path.join(root_path, i) for i in image_id]

for index, img_id in enumerate(image_id):
    temp_anns = []

    temp = images_df[f'{img_id}']
    words = temp.loc['words']

    for key in words.keys():
        temp_ann = {}
        temp_ann['image_id'] = img_id 
        temp_ann['id'] = index

        word = words[key]

        temp_ann['bbox'] = word['points']
        temp_anns.append(temp_ann)
    bboxes.append(temp_anns)

img_name_list = [img.split("/")[-1][:-4] for img in fnames]

def showimg(idx):
    fig, ax = plt.subplots(1, 2, dpi=512)
    img = io.imread(fnames[idx])

    anns = bboxes[idx]
    
    ax[0].imshow(img)
    ax[1].imshow(img)

    for ann in anns:
        ann_id = ann['id']
        #class_idx = ann['category_id']
        
        ax[0].set_title('original', fontsize = 7)
        ax[1].set_title(f"{img_name_list[idx]}", fontsize = 7)
        
        ax[0].set_xticks([])
        ax[0].set_yticks([])
        
        ax[1].set_xticks([])
        ax[1].set_yticks([])
        
        for pos in ['right', 'top', 'bottom', 'left']:
            ax[0].spines[pos].set_visible(False)
            ax[1].spines[pos].set_visible(False)
            
        points = np.array(ann['bbox'])
        ax[1].add_patch(patches.Polygon(
            points,
            closed = True,
            edgecolor = 'orange',
            fill = False,
            linewidth = 0.3
        ))

In [ ]:
for i in [8,9,12,14,16,18,25,26,27,29,32,37,41,42,43,44,45,51,52,53,55,62,63,69,84,92,93,97]: # index 0~99
    showimg(i)